## Agents
- Agents are systems that use an LLM as a reasoning engine to determine which actions to take and what the inputs to those actions should be.
- The results of those actions can then be fed back into the agent and it determines whether more actions are needed, or whether it is okay to finish.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv("./../.env")

True

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_community.tools import TavilySearchResults

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_ollama import OllamaEmbeddings

In [3]:
# Llama model
base_url = "http://localhost:11434"
model = "llama3.2"
llm = ChatOllama(model=model, base_url=base_url)

In [4]:
@tool
def search(query: str) -> str:
  """
  Search the web for realtime and latest information.
  for examples, news, stock market, weather updates etc.

  Args:
  query: The search query
  """
  
  search = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
  )
  
  response = search.invoke(query)
  return response


In [5]:
embedding_model = "nomic-embed-text"
embeddings = OllamaEmbeddings(model=embedding_model, base_url=base_url)

db_path = "./../store/db_supplements"
vector_store = FAISS.load_local(folder_path=db_path,
                                embeddings=embeddings,
                                allow_dangerous_deserialization=True)

# Convert vector store to retriever
retriever = vector_store.as_retriever(search_type="similarity",
                                      search_kwargs={'k': 3})

In [6]:
@tool
def health_supplements(query: str) -> str:
  """
  Search for information about Health Supplements.
  For any questions about Health and Gym Supplements, you must use this tool!,
  """
  
  response = retriever.invoke(query)
  return response

In [7]:
health_supplements.invoke("What is the best supplement for muscle gain?")

[Document(metadata={'source': '.././data/rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'file_path': '.././data/rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'page': 0, 'total_pages': 15, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113850Z', 'trapped': ''}, page_content='acids than traditional protein sources. Its numerous benefits have made it a popular choice\nfor snacks and drinks among consumers [3]. Another widely embraced supplement is\ncaffeine, which is found in many sports and food supplements. Caffeine reduces perceived\neffort, minimizes fatigue and pain, and proves to be effective for endurance and high-\nintensity activities, which is the choice of consumers [4].\nCreatine monohydrate is another well-known supplement used to gain muscle mass\nand support performance and recovery. It is known not to 

### Agent Creation
 - The agent is responsible for taking in input and deciding what actions to take.
 - the Agent does not execute those actions - that is done by the AgentExecutor.
 - `create_tool_calling_agent` will call `.bind_tools` for us under the hood.

In [8]:
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [10]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [11]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [13]:
tools = [search, health_supplements]
agent = create_tool_calling_agent(llm, tools, prompt)

In [14]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
question = "What is the best supplement for muscle gain?"
response = agent_executor.invoke({"input": question})



> Entering new AgentExecutor chain...

Invoking: `health_supplements` with `{'query': 'best supplements for muscle gain'}`


[Document(metadata={'source': '.././data/rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'file_path': '.././data/rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'page': 1, 'total_pages': 15, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113850Z', 'trapped': ''}, page_content='Foods 2024, 13, 1424\n2 of 21\nand sports industry, evidence suggests that creatine can benefit not only athletes but also\nthe elderly and the general population [6]. Branched-chain amino acids (BCAA) also offer\na plethora of benefits for consumers. As explained by Sanz et al. [7], BCAAs are stored\ndirectly in muscles and serve as the raw materials needed to build new muscle. This\ncontributes to the overall process o

In [22]:
question = "How is the situation in Iran?"
response = agent_executor.invoke({"input": question})



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': 'Iran current situation'}`


[{'url': 'https://www.newsnow.com/us/World/Middle+East/Iran', 'content': 'Iran, a nation of over 85 million people, continues to play a significant role in global geopolitics. The country is currently led by President Masoud Pezeshkian, who took office on 28 July 2024, succeeding Ebrahim Raisi. Iran\'s Supreme Leader, Ayatollah Ali Khamenei, remains the ultimate authority in the country\'s unique political system that combines elements of theocracy and republic. [...] Economically, Iran continues to grapple with the effects of long-standing international sanctions. The country has been pursuing a "resistance economy" policy, focusing on domestic production and reducing reliance on oil exports. Despite these efforts, inflation and unemployment remain significant challenges for the Iranian population. [...] Domestically, Iran has seen periodic waves of protests, with citizens expressi

In [23]:
response

{'input': 'How is the situation in Iran?',
 'output': "The current situation in Iran is complex and dynamic. The country continues to face significant challenges, including economic hardships, political restrictions, and social issues. The ongoing debate within Iran about the pace and extent of social and political reforms adds to the complexity.\n\nRecently, Iran was involved in a 12-day conflict with Israel, which had a devastating impact on the country's economy and infrastructure. The war has led to widespread destruction, power outages, and shortages of essential goods and services.\n\nInternationally, Iran's position has become more vulnerable, particularly regarding its nuclear and missile programs. The US and European countries have been demanding the dismantling of these programs, which has weakened Iran's leverage in international negotiations.\n\nDomestically, the Iranian government is facing significant challenges, including a rise in executions, with over 900 recorded case